In [1]:
''' 
RUSBOOST BINARY
'''
from sklearn.metrics import  accuracy_score, confusion_matrix,precision_score, recall_score, f1_score,roc_auc_score,roc_curve,precision_recall_curve, average_precision_score
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV,RepeatedStratifiedKFold
from   Models import RusBoost 
from sklearn.model_selection import StratifiedKFold

'''
This return  the features found during the training
'''
import pandas as pd
def FoundFeatures(Path):
    # reading the found dataset for the training
    df = pd.read_excel(Path)
    Features = set(df.columns)
    return Features

# removing the id and the subtypes of a given dataset
def extract_PATIENT_INFO(db):
    ids =  db.pop("ID")
    morf_codificata = db.pop("morf_codificata")
    luogoTC = db.pop("luogoTC_codificato")
    return db, ids, morf_codificata, luogoTC

In [2]:
db = pd.read_excel("Dataset/Cleaned_dataset.xlsx")

 # extrating the label
y = db.pop("maligno")
X = db

# configure the cross-validation procedure
cv_outer = StratifiedKFold(n_splits=5,shuffle=True, random_state=42)
# enumerate splits
outer_results = list()
best_params_per_fold = []
scores = pd.DataFrame(columns=['Outer Fold',"Grid-Search",'F1-score', 'Recall', 'Precision', 'Accuracy', 'Auc-Score', '#Features'])

train_folds = []
test_folds = []

for index, (train_ix, test_ix) in enumerate(cv_outer.split(X, y)):
    # split data
    X_train, X_test = X.iloc[train_ix, :], X.iloc[test_ix, :]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    
    train_folds.append(X_train.copy())
    test_folds.append(X_test.copy())

    # configure the cross-validation procedure
    cv_inner = StratifiedKFold(n_splits=5,shuffle = True, random_state=42)
    
    pipeline = RusBoost.Define_Pipeline()


    X_train, ids, morf_codificata, luogoTC_train = extract_PATIENT_INFO(X_train)

    Grid_searchOBJ = RusBoost.Grid_Search(pipeline=pipeline, cv = cv_inner,  X_train=X_train, y_train=y_train)

    best_params = Grid_searchOBJ.best_params_
    best_params_per_fold.append(Grid_searchOBJ.best_params_)
    print(f"Fold {len(best_params_per_fold)}: Best Params = {Grid_searchOBJ.best_params_}")

    ##### Elements/RusBoost/AcrossSplits

    Chosen_pipeline = RusBoost.Define_Pipeline()

    Chosen_pipeline.set_params(**Grid_searchOBJ.best_params_)
    fitted_pipeline = RusBoost.fit(pipeline=Chosen_pipeline, X_train=X_train, y_train=y_train, Prefix="AcrossSplits/Split{}_".format(index))
    X_test, ids_test, morf_codificata_test, luogoTC_test = extract_PATIENT_INFO(X_test)
    
    predictions = fitted_pipeline.predict(X_test)
    probs = fitted_pipeline.predict_proba(X_test)
    
    ###Metrics
    Accuracy = round(accuracy_score(y_test, predictions),3)
    precision = round(precision_score(y_test, predictions),3)
    recall = round(recall_score(y_test, predictions),3)
    f1 = round(f1_score(y_test, predictions),3)
    probs = probs[:, 1]
    auc_roc = round(roc_auc_score(y_test,probs),3)
    print("TEST TIME F1 is ", f1)
    ##Features
    AllFeaturesRusBoost = FoundFeatures("./Elements/RusBoost/AcrossSplits/Split{}_RusBoost_ReduceDataset.xlsx".format(index))
    print("it has found {} features".format(len(AllFeaturesRusBoost)))

    new_row = [index, Grid_searchOBJ.best_score_, f1, recall, precision, Accuracy, auc_roc, len(AllFeaturesRusBoost)]  
    scores.loc[index] = new_row  # Append the new row
    
scores['Outer Fold'] = scores['Outer Fold'].astype(int)
scores["#Features"] = scores["#Features"].astype(int)
scores.to_csv('Elements/RusBoost/AcrossSplits/scores.csv', index=False)  # index=False to avoid saving the row numbers
scores


[2024-11-08 17:46:40,593: INFO: RusBoost: Defining the pipeline...]
[2024-11-08 17:46:40,596: INFO: RusBoost: Let's define the cross validation:StratifiedKFold(n_splits=5, random_state=42, shuffle=True)]
[2024-11-08 17:46:40,596: INFO: RusBoost: Let's define the parameters for the grid search: {'rusboostclassifier__n_estimators': [150, 300], 'rusboostclassifier__learning_rate': [0.05, 0.01]}]
Best: 0.789845 using {'rusboostclassifier__learning_rate': 0.05, 'rusboostclassifier__n_estimators': 300}
Fold 1: Best Params = {'rusboostclassifier__learning_rate': 0.05, 'rusboostclassifier__n_estimators': 300}
[2024-11-08 17:46:50,155: INFO: RusBoost: Defining the pipeline...]
Time taken to fit the model: 1.5167 seconds
[2024-11-08 17:46:53,029: INFO: RusBoost: saving the resulting dataframe into: RusBoost_ReduceDataset.xlsx]
TEST TIME F1 is  0.771
it has found 124 features
[2024-11-08 17:46:53,749: INFO: RusBoost: Defining the pipeline...]
[2024-11-08 17:46:53,750: INFO: RusBoost: Let's define

,Outer Fold,Grid-Search,F1-score,Recall,Precision,Accuracy,Auc-Score,#Features
0,0,0.789845,0.771,0.881,0.685,0.895,0.963,124
1,1,0.811912,0.804,0.951,0.696,0.909,0.974,124
2,2,0.802535,0.831,0.881,0.787,0.928,0.976,124
3,3,0.808185,0.720,0.857,0.621,0.866,0.961,124
4,4,0.787072,0.792,0.952,0.678,0.900,0.964,124


In [3]:
best_params_per_fold

[{'rusboostclassifier__learning_rate': 0.05,
  'rusboostclassifier__n_estimators': 300},
 {'rusboostclassifier__learning_rate': 0.05,
  'rusboostclassifier__n_estimators': 300},
 {'rusboostclassifier__learning_rate': 0.05,
  'rusboostclassifier__n_estimators': 300},
 {'rusboostclassifier__learning_rate': 0.01,
  'rusboostclassifier__n_estimators': 300},
 {'rusboostclassifier__learning_rate': 0.05,
  'rusboostclassifier__n_estimators': 150}]

In [4]:
import numpy as np
def compute_stats(scores):
    cols = scores.columns[1:-1]
    mean_row_values = scores[cols].mean()
    std_row_values = scores[cols].std()
    
    # Create new rows with NaNs for the columns that will be replaced
    mean_row = pd.Series("", index=scores.columns, name='mean')
    std_row = pd.Series("", index=scores.columns, name='std')
    mean_row[cols] = mean_row_values.round(3)
    std_row[cols] = std_row_values.round(3)
    
    # Convert Series to DataFrame
    mean_df = mean_row.to_frame().T
    std_df = std_row.to_frame().T
    
    scores_new = pd.concat([scores, mean_df, std_df], axis = 0)
    
    scores_new.loc["mean","#Features"] = str(int(round(np.mean(scores_new["#Features"][:-2]),0)))
    scores_new.loc["std","#Features"] = str(int(round(np.std( scores_new["#Features"][:-2]),0)))
    
    scores_new.to_csv('Elements/RusBoost/AcrossSplits/scores.csv', index=False)  # index=False to avoid saving the row numbers
    
    return scores_new

scores_agg = compute_stats(scores)
scores_agg

,Outer Fold,Grid-Search,F1-score,Recall,Precision,Accuracy,Auc-Score,#Features
0,0,0.789845,0.771,0.881,0.685,0.895,0.963,124
1,1,0.811912,0.804,0.951,0.696,0.909,0.974,124
2,2,0.802535,0.831,0.881,0.787,0.928,0.976,124
3,3,0.808185,0.72,0.857,0.621,0.866,0.961,124
4,4,0.787072,0.792,0.952,0.678,0.9,0.964,124
mean,,0.8,0.784,0.904,0.693,0.9,0.968,124
std,,0.011,0.042,0.044,0.06,0.023,0.007,0


In [5]:
import matplotlib.pyplot as plt
def StackedHist(df):
    # Aggregate counts by Center and Morf_Codificata
    counts = df.groupby(['luogoTC_codificato', 'morf_codificata']).size().unstack(fill_value=0)

    # Define all possible center labels (1 to 14)
    all_centers = list(range(1, 15))
    all_labels = sorted(set(map(int,df['morf_codificata'])))
    
    # Reindex the DataFrame to include all centers, filling missing values with 0
    counts = counts.reindex(index=all_centers, columns=all_labels, fill_value=0)

    # Create the stacked histogram
    fig, ax = plt.subplots(figsize=(12, 10))

    # Initialize bottom positions for stacking
    bottom = np.zeros(len(all_centers))
    
    
    colors = plt.cm.tab20.colors



    # Plot each Morf_Codificata category as a stacked bar segment
    for i, label in enumerate(all_labels):
        ax.bar(all_centers, counts[label], bottom=bottom, label=f'{label}', color=colors[i])
        bottom += counts[label].values  # Update bottom for next stack

    # Set x-ticks from 1 to 14
    ax.set_xticks(all_centers)
    ax.set_xticklabels(all_centers, rotation=45, ha='right')

    # Adding labels and title
    plt.xlabel('Center')
    plt.ylabel('Count')
    plt.title('Stacked Histogram by Center and Morf_Codificata')
    plt.legend(title='Morf_Codificata', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()